# Python Notebook based utility to Audit the VLM based Progress tracker

In [ ]:
import os
import json
from groundedvision.config import RAW_DATA_DIR, PROCESSED_DATA_DIR
from loguru import logger

In [ ]:
path = f'{PROCESSED_DATA_DIR}/matched_pairs_json'
list_of_aligned_directories = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

In [ ]:
from pathlib import Path

qwen_compare_results = {}
no_changes = []
for folder_path in list_of_aligned_directories:
    input_folder_path = f"{PROCESSED_DATA_DIR}/matched_pairs_json/{folder_path}/cubemap_cross/"
    final_output_path = f"{PROCESSED_DATA_DIR}/matched_pairs_json/{folder_path}/construction_progress_report_qwen3_vl_plus.json"

    if not os.path.exists(final_output_path):
        continue

    for image_path in os.listdir(input_folder_path):
        output_path = f"{PROCESSED_DATA_DIR}/matched_pairs_json/{folder_path}"
        full_path = os.path.join(input_folder_path, image_path)
        path_full_path = Path(full_path)
        # check if the image patch contains
        if "cubemap_cross_collage.jpg" in full_path:
            filename = path_full_path.stem
            if filename.startswith("new_"):
                stripped = filename[4:]  # Remove "new_"
                new_panorama_frame = stripped.split("_aligned")[0]
            if filename.startswith("old_"):
                stripped = filename[4:]  # Remove "old_"
                old_panorama_frame = stripped.split("_aligned")[0]
    
    report = None

    with open(final_output_path, 'r') as f:
        report = json.load(f)
    
    if report:
        print(folder_path, final_output_path)
        if not report['structural_change_detected']:
            no_changes.append(folder_path)
        print(new_panorama_frame, old_panorama_frame)
        report["new_panorama_frame"] = new_panorama_frame
        report["old_panorama_frame"] = old_panorama_frame
        qwen_compare_results[folder_path] = report

In [ ]:
print(len(qwen_compare_results))
print(len(no_changes))

In [ ]:
# Cell 1: Setup and Start Audit Session
from groundedvision.audit import AnnotationManager, create_audit_session

# Path to store annotations (will be created if doesn't exist)
annotations_file = f"{PROCESSED_DATA_DIR}/audit_annotations.json"

# Create the manager - this loads existing annotations if file exists
manager = AnnotationManager(annotations_file)

# Start session - this prompts for auditor alias
manager.start_session()

In [ ]:

for folder_name in qwen_compare_results:
    report = qwen_compare_results[folder_name]
    new_panorama_frame = report['new_panorama_frame']
    old_panorama_frame = report['old_panorama_frame']

    collage_new = f"{PROCESSED_DATA_DIR}/matched_pairs_json/{folder_path}/new_{new_panorama_frame}_aligned_cube_map/collage_border.png"
    collage_old = f"{PROCESSED_DATA_DIR}/matched_pairs_json/{folder_path}/old_{old_panorama_frame}_aligned_cube_map/collage_border.png"
    print(collage_old)
    break





# Cell 2: Annotate samples as you review them
# For each sample you audit:
manager.add_annotation(
    sample_id="old_VID_20251210_123543_00_325_IMG_00650",
    verdict="correct",  # or "incorrect", "partial", "skip"
    notes="Model correctly identified wall changes",
    confidence=0.9,
    model_output={"changes_detected": True, "confidence": 0.85}
)